In [1]:
import librosa
from fp import featurepoint
from glob import glob
import soundfile as sf

from attacks import *
from stats import *

攻击器生成

In [2]:
clp = Closed_loop()
noise_20dB = Noise(20)
noise_15dB = Noise(15)
noise_10dB = Noise(10)
amp_13 = Amplitude(1.3)
amp_07 = Amplitude(0.7)
requan = Requanzation()
resam_22k = Resample(22050)
resam_11k = Resample(11025)
resam_8k = Resample(8000)
mp3_128 = Mp3_compression(128)
mp3_96 = Mp3_compression(96)
lpf = Lowpass(8000)
lpf_5k = Lowpass(5000)
crop_10_10 = Cropping(10, 10)
crop_5_20 = Cropping(5, 20)

atks = [clp, noise_20dB, noise_15dB, noise_10dB, amp_13, amp_07, requan, resam_22k, resam_11k, resam_8k, mp3_128, mp3_96, lpf, lpf_5k, crop_10_10, crop_5_20]

for atk in atks:
    with open(os.path.join('./logfile/', atk.name + '.csv'), 'a+') as f:
        f.write('filename,precision,recall\n')

端到端特征点提取

In [3]:
def fileidx(f):
    sr = librosa.get_samplerate(f)
    x, sr = librosa.load(f, sr)
    return featurepoint(x, 1.5 * sr)

In [4]:
filename = []
filename += glob('./data/*.mp3')

for fil in filename:
    basename, ext = os.path.splitext(fil)
    x, sr = librosa.load(fil, sr=librosa.get_samplerate(fil))
    sf.write(basename+'.wav', x.T, sr)
    os.remove(fil)

files = []
files += glob('./data/*.wav')

In [5]:
for f in files:
    original = fileidx(f)
    for atk in atks:
        file = atk(f)
        test = fileidx(file)
        if atk.name[:4] != "_cro":
            precision, recall = precision_recall(test, original)
            with open(os.path.join('./logfile/', atk.name + '.csv'), 'a+') as csv:
                csv.write("{}, {}, {}\n".format(f, precision, recall))
        else:
            sr = librosa.get_samplerate(file)
            region = (original< (atk.ss + atk.duration) * sr) & (original > atk.ss * sr)
            origin = original[region] - (atk.ss * sr)
            precision, recall = precision_recall(test, origin)
            with open(os.path.join('./logfile/', atk.name + '.csv'), 'a+') as csv:
                csv.write("{}, {}, {}\n".format(f, precision, recall))
        
        if atk.name != '_none':
            os.remove(file)

In [6]:
import pandas as pd

with open('./logfile/mean.csv', 'a+') as f:
    f.write("name,precision,recall\n")

In [7]:
for atk in atks:
    df = pd.read_csv(os.path.join('./logfile', atk.name + '.csv'))
    with open('./logfile/mean.csv', 'a+') as f:
        f.write('{},{},{}\n'.format(atk.name, df['precision'].mean(), df['recall'].mean()))